<a href="https://colab.research.google.com/github/RiazMahmood6/E-book/blob/main/E_book_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install openai

import openai
from google.colab import userdata
import time

# Set up OpenAI API key (store your API key in Colab secrets)
# Go to "Secrets" tab in Colab (lock icon on left) and add 'OPENAI_API_KEY' with your key
openai.api_key = userdata.get('OPENAI_API_KEY')

class BookGenerator:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)

    def generate_chapter(self, title, description, chapter_num, prev_content=""):
        """Generate a single chapter based on book details and previous content"""
        prompt = f"""
        You are an expert author writing a book titled '{title}'.
        The book description is: '{description}'.
        Write Chapter {chapter_num} of this book.
        Make it approximately 1000 words long.
        Ensure it follows a logical story progression.
        Here's the previous content to maintain continuity (if any): {prev_content[:1000]}

        Format the output as:
        # Chapter {chapter_num}
        [chapter content]
        """

        response = self.client.chat.completions.create(
            model="gpt-4-turbo",  # You can change to your preferred model
            messages=[
                {"role": "system", "content": "You are a creative writer specializing in coherent, engaging stories."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=1500,
            temperature=0.7
        )

        return response.choices[0].message.content

    def generate_book(self, title, description, num_chapters=5):
        """Generate a complete book with multiple chapters"""
        print(f"Generating book: {title}")
        print(f"Description: {description}\n")

        book_content = []
        prev_content = ""

        for chapter in range(1, num_chapters + 1):
            print(f"Generating Chapter {chapter}...")
            chapter_content = self.generate_chapter(title, description, chapter, prev_content)
            book_content.append(chapter_content)
            prev_content = chapter_content
            time.sleep(2)  # Rate limiting

        return book_content

    def save_book(self, title, content_list):
        """Save the book to a text file"""
        filename = f"{title.replace(' ', '_')}.txt"
        with open(filename, 'w') as f:
            f.write(f"# {title}\n\n")
            for chapter in content_list:
                f.write(chapter)
                f.write("\n\n")
        return filename

# Example usage
def main():
    # Initialize the generator
    book_gen = BookGenerator(openai.api_key)

    # Book details
    book_title = "The Quantum Chronicles"
    book_description = "A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning multiple dimensions."
    num_chapters = 3  # Adjust as needed

    # Generate the book
    book_content = book_gen.generate_book(book_title, book_description, num_chapters)

    # Save the book
    filename = book_gen.save_book(book_title, book_content)
    print(f"\nBook saved as: {filename}")

    # Optional: Display first 500 characters of each chapter
    for i, chapter in enumerate(book_content, 1):
        print(f"\nChapter {i} preview:")
        print(chapter[:500] + "...")

if __name__ == "__main__":
    main()

# To download the file in Colab
from google.colab import files
files.download('The_Quantum_Chronicles.txt')

Generating book: The Quantum Chronicles
Description: A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning multiple dimensions.

Generating Chapter 1...
Generating Chapter 2...
Generating Chapter 3...

Book saved as: The_Quantum_Chronicles.txt

Chapter 1 preview:
# Chapter 1

Dr. Elara Myles adjusted the straps on her leather wristwatch, a nervous habit she had developed over the years whenever her mind wrestled with the enormity of her work. Today, she stood before the culmination of her life’s research: the ChronoSphere. A gleaming orb of interconnected silver rings and pulsating blue light, it hummed softly in the center of the lab, a symphony of potential energy waiting to be unleashed.

“Are we ready, Dr. Myles?” asked a voice from the observation d...

Chapter 2 preview:
# Chapter 2

The ChronoSphere's core hummed louder, its rings spinning faster until they were just a blur. The blue glow intensified, casting eerie shadows across the lab's s

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install required libraries
!pip install openai reportlab

import openai
from google.colab import userdata
import time
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch

# Set up OpenAI API key (store your API key in Colab secrets)
openai.api_key = userdata.get('OPENAI_API_KEY')

class BookGenerator:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        self.styles = getSampleStyleSheet()

    def generate_chapter(self, title, description, chapter_num, target_words, prev_content=""):
        """Generate a single chapter with specified word count"""
        prompt = f"""
        You are an expert author writing a book titled '{title}'.
        The book description is: '{description}'.
        Write Chapter {chapter_num} of this book.
        Make it approximately {target_words} words long.
        Ensure it follows a logical story progression.
        Here's the previous content to maintain continuity (if any): {prev_content[:1000]}

        Format the output as:
        # Chapter {chapter_num}
        [chapter content]
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo",  # Change as needed
                messages=[
                    {"role": "system", "content": "You are a creative writer specializing in coherent, engaging stories."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=int(target_words / 0.75),  # Fixed: Convert to integer
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error generating Chapter {chapter_num}: {str(e)}")
            return f"# Chapter {chapter_num}\n[Chapter generation failed due to API error]"

    def generate_cover_prompt(self, title, description):
        """Generate a prompt for cover image description"""
        return f"""
        Create a detailed description of a book cover for a book titled '{title}'.
        The book description is: '{description}'.
        Describe a vivid, professional book cover image that captures the essence of the story.
        Include colors, main elements, and style.
        """

    def generate_book(self, title, description, target_pages=50):
        """Generate a complete book aiming for target page count"""
        words_per_page = 250
        total_words = target_pages * words_per_page
        words_per_chapter = 2000
        num_chapters = max(1, total_words // words_per_chapter)

        print(f"Generating book: {title}")
        print(f"Description: {description}")
        print(f"Target pages: {target_pages}, Estimated chapters: {num_chapters}\n")

        book_content = []
        prev_content = ""

        for chapter in range(1, num_chapters + 1):
            print(f"Generating Chapter {chapter}...")
            chapter_content = self.generate_chapter(title, description, chapter, words_per_chapter, prev_content)
            book_content.append(chapter_content)
            prev_content = chapter_content
            time.sleep(2)  # Rate limiting

        # Generate cover description
        try:
            cover_prompt = self.generate_cover_prompt(title, description)
            cover_response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[{"role": "user", "content": cover_prompt}]
            )
            cover_description = cover_response.choices[0].message.content
        except Exception as e:
            print(f"Error generating cover description: {str(e)}")
            cover_description = "[Cover description generation failed]"

        return book_content, cover_description

    def save_book_as_pdf(self, title, content_list, cover_description):
        """Save the book as a PDF with a cover page"""
        filename = f"{title.replace(' ', '_')}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Cover page
        story.append(Paragraph(f"<font size=24>{title}</font>", self.styles['Title']))
        story.append(Spacer(1, 0.5 * inch))
        story.append(Paragraph("Cover Description:", self.styles['Heading2']))
        story.append(Paragraph(cover_description, self.styles['BodyText']))
        story.append(Spacer(1, 1 * inch))

        # Add chapters
        for chapter in content_list:
            lines = chapter.split('\n')
            for line in lines:
                if line.startswith('# Chapter'):
                    story.append(Paragraph(line, self.styles['Heading1']))
                else:
                    story.append(Paragraph(line, self.styles['BodyText']))
            story.append(Spacer(1, 0.25 * inch))

        try:
            doc.build(story)
            print(f"PDF successfully created: {filename}")
        except Exception as e:
            print(f"Error creating PDF: {str(e)}")

        return filename

# Example usage
def main():
    # Initialize the generator
    book_gen = BookGenerator(openai.api_key)

    # Book details
    book_title = "The Quantum Chronicles"
    book_description = "A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning multiple dimensions."
    target_pages = 50  # Set up to 500 as needed

    # Generate the book and cover
    book_content, cover_description = book_gen.generate_book(book_title, book_description, target_pages)

    # Save as PDF
    filename = book_gen.save_book_as_pdf(book_title, book_content, cover_description)
    print(f"\nBook saved as: {filename}")

    # Display cover description
    print("\nGenerated Cover Description:")
    print(cover_description)

    # Note about image generation
    print("\nNote: To generate an actual cover image, confirm if you'd like me to proceed with image generation based on the description above.")

if __name__ == "__main__":
    main()

# Download the PDF in Colab
from google.colab import files
files.download('The_Quantum_Chronicles.pdf')

Generating book: The Quantum Chronicles
Description: A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning multiple dimensions.
Target pages: 50, Estimated chapters: 6

Generating Chapter 1...
Generating Chapter 2...
Generating Chapter 3...
Generating Chapter 4...
Generating Chapter 5...
Generating Chapter 6...
PDF successfully created: The_Quantum_Chronicles.pdf

Book saved as: The_Quantum_Chronicles.pdf

Generated Cover Description:
For the cover of "The Quantum Chronicles," the imagery needs to evoke a sense of expansive depth, centering around the themes of time-travel and multi-dimensional exploration. The cover is a blend of abstract and literal elements, using a saturated palette to suggest both the mysterious and the scientific aspects of the narrative.

**Main Imagery:**
At the heart of the cover is an image of our protagonist, a slightly disheveled but compelling physicist, peering curiously into a swirling vortex that occupies the centra

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install required libraries
!pip install openai reportlab ipywidgets

import openai
from google.colab import userdata
import time
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import ipywidgets as widgets
from IPython.display import display

# Set up OpenAI API key (store your API key in Colab secrets)
openai.api_key = userdata.get('OPENAI_API_KEY')

class BookGenerator:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        self.styles = getSampleStyleSheet()

    def generate_chapter(self, title, description, chapter_num, target_words, language="en", prev_content=""):
        """Generate a single chapter with specified word count and language"""
        lang_prompt = f"Write the chapter in {language} (ISO code: {language})." if language != "en" else ""
        prompt = f"""
        You are an expert author writing a book titled '{title}'.
        The book description is: '{description}'.
        Write Chapter {chapter_num} of this book.
        Make it approximately {target_words} words long.
        Ensure it follows a logical story progression.
        {lang_prompt}
        Here's the previous content to maintain continuity (if any): {prev_content[:1000]}

        Format the output as:
        # Chapter {chapter_num}
        [chapter content]
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo",  # Change as needed
                messages=[
                    {"role": "system", "content": "You are a creative writer specializing in coherent, engaging stories."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=int(target_words / 0.75),
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error generating Chapter {chapter_num}: {str(e)}")
            return f"# Chapter {chapter_num}\n[Chapter generation failed due to API error]"

    def generate_cover_prompt(self, title, description, language="en"):
        """Generate a prompt for cover image description in specified language"""
        lang_prompt = f"Write the description in {language} (ISO code: {language})." if language != "en" else ""
        return f"""
        Create a detailed description of a book cover for a book titled '{title}'.
        The book description is: '{description}'.
        Describe a vivid, professional book cover image that captures the essence of the story.
        Include colors, main elements, and style.
        {lang_prompt}
        """

    def generate_book(self, title, description, target_pages=10, language="en"):
        """Generate a complete book aiming for target page count in specified language"""
        words_per_page = 250
        total_words = target_pages * words_per_page
        words_per_chapter = 2000
        num_chapters = max(1, total_words // words_per_chapter)

        print(f"Generating book: {title}")
        print(f"Description: {description}")
        print(f"Language: {language}")
        print(f"Target pages: {target_pages}, Estimated chapters: {num_chapters}\n")

        book_content = []
        prev_content = ""

        for chapter in range(1, num_chapters + 1):
            print(f"Generating Chapter {chapter}...")
            chapter_content = self.generate_chapter(title, description, chapter, words_per_chapter, language, prev_content)
            book_content.append(chapter_content)
            prev_content = chapter_content
            time.sleep(2)  # Rate limiting

        # Generate cover description
        try:
            cover_prompt = self.generate_cover_prompt(title, description, language)
            cover_response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[{"role": "user", "content": cover_prompt}]
            )
            cover_description = cover_response.choices[0].message.content
        except Exception as e:
            print(f"Error generating cover description: {str(e)}")
            cover_description = "[Cover description generation failed]"

        return book_content, cover_description

    def save_book_as_pdf(self, title, content_list, cover_description):
        """Save the book as a PDF with a cover page"""
        filename = f"{title.replace(' ', '_')}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Cover page
        story.append(Paragraph(f"<font size=24>{title}</font>", self.styles['Title']))
        story.append(Spacer(1, 0.5 * inch))
        story.append(Paragraph("Cover Description:", self.styles['Heading2']))
        story.append(Paragraph(cover_description, self.styles['BodyText']))
        story.append(Spacer(1, 1 * inch))

        # Add chapters
        for chapter in content_list:
            lines = chapter.split('\n')
            for line in lines:
                if line.startswith('# Chapter'):
                    story.append(Paragraph(line, self.styles['Heading1']))
                else:
                    story.append(Paragraph(line, self.styles['BodyText']))
            story.append(Spacer(1, 0.25 * inch))

        try:
            doc.build(story)
            print(f"PDF successfully created: {filename}")
        except Exception as e:
            print(f"Error creating PDF: {str(e)}")

        return filename

# Language selection widget
language_options = {
    "English": "en",
    "Bangla": "bn",
    "Spanish": "es",
    "French": "fr",
    "Hindi": "hi",
    "Arabic": "ar",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    "German": "de",
    "Japanese": "ja",
    "Custom (Enter ISO 639-1 code)": "custom"
}

language_dropdown = widgets.Dropdown(
    options=language_options.keys(),
    value="English",
    description="Language:"
)

custom_language_input = widgets.Text(
    value="",
    placeholder="Enter ISO 639-1 code (e.g., 'bn' for Bangla)",
    description="Custom Code:",
    disabled=True
)

title_input = widgets.Text(
    value="The Quantum Chronicles",
    description="Book Title:"
)

description_input = widgets.Textarea(
    value="A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning multiple dimensions.",
    description="Description:",
    layout={'width': '400px', 'height': '100px'}
)

pages_input = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description="Target Pages:"
)

generate_button = widgets.Button(
    description="Generate Book",
    button_style="success"
)

def on_language_change(change):
    if change['new'] == "Custom (Enter ISO 639-1 code)":
        custom_language_input.disabled = False
    else:
        custom_language_input.disabled = True

language_dropdown.observe(on_language_change, names='value')

def generate_book_clicked(b):
    book_gen = BookGenerator(openai.api_key)
    title = title_input.value
    description = description_input.value
    target_pages = pages_input.value

    if language_dropdown.value == "Custom (Enter ISO 639-1 code)":
        language = custom_language_input.value if custom_language_input.value else "en"
    else:
        language = language_options[language_dropdown.value]

    book_content, cover_description = book_gen.generate_book(title, description, target_pages, language)
    filename = book_gen.save_book_as_pdf(title, book_content, cover_description)

    print(f"\nBook saved as: {filename}")
    print("\nGenerated Cover Description:")
    print(cover_description)
    print("\nNote: To generate an actual cover image, confirm if you'd like to proceed with image generation based on the description above.")

    from google.colab import files
    files.download(filename)

generate_button.on_click(generate_book_clicked)

# Display widgets
display(language_dropdown, custom_language_input, title_input, description_input, pages_input, generate_button)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.0 MB/s eta 0:00:00


Dropdown(description='Language:', options=('English', 'Bangla', 'Spanish', 'French', 'Hindi', 'Arabic', 'Chine…

Text(value='', description='Custom Code:', disabled=True, placeholder="Enter ISO 639-1 code (e.g., 'bn' for Ba…

Text(value='The Quantum Chronicles', description='Book Title:')

Textarea(value='A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning mu…

IntSlider(value=50, description='Target Pages:', max=500, min=10, step=10)

Button(button_style='success', description='Generate Book', style=ButtonStyle())

Generating book: শিশু কিশোরদের পদার্থবিজ্ঞান
Description: শিশু কিশোরদের পদার্থবিজ্ঞান
Language: bn
Target pages: 20, Estimated chapters: 2

Generating Chapter 1...
Generating Chapter 2...
PDF successfully created: শিশু_কিশোরদের_পদার্থবিজ্ঞান.pdf

Book saved as: শিশু_কিশোরদের_পদার্থবিজ্ঞান.pdf

Generated Cover Description:
বইয়ের প্রচ্ছদটি বিস্ময়কর এবং উজ্জ্বল রংয়ের মিশ্রণে সাজানো হয়েছে—মূলত বেগুনি, নীল এবং হলুদ রং ব্যবহৃত হয়েছে। প্রচ্ছদের মুখ্য আকর্ষণ হলো একটি মহাকাশ যান যা মহাকাশে ভেসে বেড়াচ্ছে, এর পাশে বিভিন্ন গ্রহ এবং তারাদের ছবি দেখা যাচ্ছে। মহাকাশ যানের চারদিকে উজ্জ্বল স্ফটিকালোক এবং তারকা কণার এক ঝলকানি আবহ তৈরি করেছে। 

প্রচ্ছদের উপরের দিকে বড় করে এবং আকর্ষণীয় ফন্টে "শিশু কিশোরদের পদার্থবিজ্ঞান" শিরোনামটি লেখা হয়েছে। শিরোনামের অক্ষরগুলো কিছুটা মেটালিক আভায় এবং নক্ষত্রের আলো প্রতিবিম্বিত হিসেবে দেখা যাচ্ছে, যা পদার্থবিজ্ঞানের উচ্চতা এবং গভীরতা প্রতিফলিত করে। 

কভারের নীচের দিকে এক ঝাঁক শিশু ও কিশোর খেলছে এমন একটি ছবি রয়েছে। তারা কিছু পরীক্ষা করছে- বিজ্ঞানের কিট দিয়ে গো

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install required libraries
!pip install openai reportlab ipywidgets requests

import openai
from google.colab import userdata
import time
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import ipywidgets as widgets
from IPython.display import display
import requests
import os

# Set up OpenAI API key (store your API key in Colab secrets)
openai.api_key = userdata.get('OPENAI_API_KEY')

class BookGenerator:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        self.styles = getSampleStyleSheet()

    def generate_chapter(self, title, description, chapter_num, target_words, language="en", prev_content=""):
        """Generate a single chapter with specified word count and language"""
        lang_prompt = f"Write the chapter in {language} (ISO code: {language})." if language != "en" else ""
        prompt = f"""
        You are an expert author writing a book titled '{title}'.
        The book description is: '{description}'.
        Write Chapter {chapter_num} of this book.
        Make it approximately {target_words} words long.
        Ensure it follows a logical story progression.
        {lang_prompt}
        Here's the previous content to maintain continuity (if any): {prev_content[:1000]}

        Format the output as:
        # Chapter {chapter_num}
        [chapter content]
        """

        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a creative writer specializing in coherent, engaging stories."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=int(target_words / 0.75),
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            print(f"Error generating Chapter {chapter_num}: {str(e)}")
            return f"# Chapter {chapter_num}\n[Chapter generation failed due to API error]"

    def generate_cover_image(self, title, description):
        """Generate a cover image using DALL-E"""
        prompt = f"""
        Create a vivid, professional book cover image for a book titled '{title}'.
        The book description is: '{description}'.
        Include a mix of abstract and realistic elements capturing the story's essence.
        Use vibrant colors and a sci-fi aesthetic if applicable.
        """
        try:
            response = self.client.images.generate(
                model="dall-e-3",  # Use "dall-e-2" if needed
                prompt=prompt,
                n=1,
                size="1024x1024"
            )
            image_url = response.data[0].url
            # Download the image
            image_path = f"{title.replace(' ', '_')}_cover.jpg"
            with open(image_path, 'wb') as f:
                f.write(requests.get(image_url).content)
            return image_path
        except Exception as e:
            print(f"Error generating cover image: {str(e)}")
            return None

    def generate_book(self, title, description, target_pages=50, language="en"):
        """Generate a complete book with cover image"""
        words_per_page = 250
        total_words = target_pages * words_per_page
        words_per_chapter = 2000
        num_chapters = max(1, total_words // words_per_chapter)

        print(f"Generating book: {title}")
        print(f"Description: {description}")
        print(f"Language: {language}")
        print(f"Target pages: {target_pages}, Estimated chapters: {num_chapters}\n")

        book_content = []
        prev_content = ""

        for chapter in range(1, num_chapters + 1):
            print(f"Generating Chapter {chapter}...")
            chapter_content = self.generate_chapter(title, description, chapter, words_per_chapter, language, prev_content)
            book_content.append(chapter_content)
            prev_content = chapter_content
            time.sleep(2)  # Rate limiting

        # Generate cover image
        cover_image_path = self.generate_cover_image(title, description)

        return book_content, cover_image_path

    def save_book_as_pdf(self, title, content_list, cover_image_path):
        """Save the book as a PDF with a cover image"""
        filename = f"{title.replace(' ', '_')}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Cover page with image
        story.append(Paragraph(f"<font size=24>{title}</font>", self.styles['Title']))
        story.append(Spacer(1, 0.5 * inch))
        if cover_image_path and os.path.exists(cover_image_path):
            img = Image(cover_image_path, width=4*inch, height=4*inch)
            story.append(img)
            story.append(Spacer(1, 0.5 * inch))
        else:
            story.append(Paragraph("Cover image generation failed", self.styles['BodyText']))
        story.append(Spacer(1, 1 * inch))

        # Add chapters
        for chapter in content_list:
            lines = chapter.split('\n')
            for line in lines:
                if line.startswith('# Chapter'):
                    story.append(Paragraph(line, self.styles['Heading1']))
                else:
                    story.append(Paragraph(line, self.styles['BodyText']))
            story.append(Spacer(1, 0.25 * inch))

        try:
            doc.build(story)
            print(f"PDF successfully created: {filename}")
        except Exception as e:
            print(f"Error creating PDF: {str(e)}")

        return filename

# Language selection widget
language_options = {
    "English": "en",
    "Bangla": "bn",
    "Spanish": "es",
    "French": "fr",
    "Hindi": "hi",
    "Arabic": "ar",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    "German": "de",
    "Japanese": "ja",
    "Custom (Enter ISO 639-1 code)": "custom"
}

language_dropdown = widgets.Dropdown(
    options=language_options.keys(),
    value="English",
    description="Language:"
)

custom_language_input = widgets.Text(
    value="",
    placeholder="Enter ISO 639-1 code (e.g., 'bn' for Bangla)",
    description="Custom Code:",
    disabled=True
)

title_input = widgets.Text(
    value="The Quantum Chronicles",
    description="Book Title:"
)

description_input = widgets.Textarea(
    value="A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning multiple dimensions.",
    description="Description:",
    layout={'width': '400px', 'height': '100px'}
)

pages_input = widgets.IntSlider(
    value=50,
    min=10,
    max=500,
    step=10,
    description="Target Pages:"
)

generate_button = widgets.Button(
    description="Generate Book",
    button_style="success"
)

def on_language_change(change):
    if change['new'] == "Custom (Enter ISO 639-1 code)":
        custom_language_input.disabled = False
    else:
        custom_language_input.disabled = True

language_dropdown.observe(on_language_change, names='value')

def generate_book_clicked(b):
    book_gen = BookGenerator(openai.api_key)
    title = title_input.value
    description = description_input.value
    target_pages = pages_input.value

    if language_dropdown.value == "Custom (Enter ISO 639-1 code)":
        language = custom_language_input.value if custom_language_input.value else "en"
    else:
        language = language_options[language_dropdown.value]

    book_content, cover_image_path = book_gen.generate_book(title, description, target_pages, language)
    filename = book_gen.save_book_as_pdf(title, book_content, cover_image_path)

    print(f"\nBook saved as: {filename}")
    from google.colab import files
    files.download(filename)

generate_button.on_click(generate_book_clicked)

# Display widgets
display(language_dropdown, custom_language_input, title_input, description_input, pages_input, generate_button)


Dropdown(description='Language:', options=('English', 'Bangla', 'Spanish', 'French', 'Hindi', 'Arabic', 'Chine…

Text(value='', description='Custom Code:', disabled=True, placeholder="Enter ISO 639-1 code (e.g., 'bn' for Ba…

Text(value='The Quantum Chronicles', description='Book Title:')

Textarea(value='A sci-fi adventure following a time-traveling physicist who discovers a conspiracy spanning mu…

IntSlider(value=50, description='Target Pages:', max=500, min=10, step=10)

Button(button_style='success', description='Generate Book', style=ButtonStyle())

Generating book: Fight Club
Description: some roadside boys create a fight club that becomes national hit
Language: en
Target pages: 10, Estimated chapters: 1

Generating Chapter 1...
PDF successfully created: Fight_Club.pdf

Book saved as: Fight_Club.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Install required libraries
!pip install reportlab ebooklib requests ipywidgets

import openai
from google.colab import userdata
import time
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak, Table
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib.units import inch
import ipywidgets as widgets
from IPython.display import display
import requests
from ebooklib import epub
import os

# Set up OpenAI API key (store your API key in Colab secrets)
openai.api_key = userdata.get('OPENAI_API_KEY')

class BookGenerator:
    def __init__(self, api_key):
        self.client = openai.OpenAI(api_key=api_key)
        self.styles = getSampleStyleSheet()

    def generate_chapter(self, title, description, chapter_num, target_words, language="en", prev_content=""):
        prompt = f"""
        You are an expert author writing a book titled '{title}'.
        The book description is: '{description}'.
        Write Chapter {chapter_num} (~{target_words} words) in {language} (ISO code: {language}).
        Ensure logical story progression. Previous content: {prev_content[:1000]}
        Format as:
        # Chapter {chapter_num}
        [content]
        """
        try:
            response = self.client.chat.completions.create(
                model="gpt-4-turbo",
                messages=[
                    {"role": "system", "content": "You are a creative writer specializing in coherent, engaging stories."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=int(target_words / 0.75),
                temperature=0.7
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"# Chapter {chapter_num}\n[Error: {str(e)}]"

    def generate_cover_image(self, title, description):
        # Update the prompt to request only a front cover image
        prompt = f"One professional image for '{title}': {description}."

        try:
            response = self.client.images.generate(
                model="dall-e-3",
                prompt=prompt,
                n=1,
                size="1024x1024"
            )
            image_url = response.data[0].url
            image_path = f"{title.replace(' ', '_')}_cover.jpg"
            with open(image_path, 'wb') as f:
                f.write(requests.get(image_url).content)
            return image_path
        except Exception as e:
            print(f"Cover image error: {str(e)}")
            return None


    def generate_book(self, title, description, target_pages=50, language="en"):
        words_per_page = 250
        total_words = target_pages * words_per_page
        words_per_chapter = 2000
        num_chapters = max(1, total_words // words_per_chapter)

        print(f"Generating '{title}' ({language}, ~{target_pages} pages, ~{num_chapters} chapters)")

        book_content = []
        prev_content = ""
        for chapter in range(1, num_chapters + 1):
            print(f"Chapter {chapter}...")
            chapter_content = self.generate_chapter(title, description, chapter, words_per_chapter, language, prev_content)
            book_content.append(chapter_content)
            prev_content = chapter_content
            time.sleep(2)

        cover_image_path = self.generate_cover_image(title, description)
        return book_content, cover_image_path

    def save_book_as_pdf(self, title, content_list, cover_image_path):
        filename = f"{title.replace(' ', '_')}.pdf"
        doc = SimpleDocTemplate(filename, pagesize=letter)
        story = []

        # Title Page: Remove title text, just add cover image
        if cover_image_path:
            title_paragraph = Paragraph(f"<font size=24>{title}</font>", self.styles['Title'])
            story.append(title_paragraph)
            story.append(Spacer(1, 0.2 * inch))
            # Calculate max dimensions with margins (1 inch on each side)
            max_width = letter[0] - 2 * inch  # 612 - 72 = 540 points
            max_height = letter[1] - 3 * inch  # 792 - 108 = 684 points, adjusted for title space
            img = Image(cover_image_path, width=max_width, height=max_height)
            img.hAlign = 'CENTER'  # Center the image
            story.append(img)
        story.append(PageBreak())  # Move to the next page after cover image

        # Table of Contents
        toc_data = [["Chapter", "Page"]]
        page_num = 2
        for i, chapter in enumerate(content_list, 1):
            toc_data.append([f"Chapter {i}", str(page_num)])
            page_num += 1
        story.append(Paragraph("Table of Contents", self.styles['Heading1']))
        story.append(Spacer(1, 0.25 * inch))
        toc_table = Table(toc_data, colWidths=[4*inch, 1*inch])
        story.append(toc_table)
        story.append(PageBreak())

        # Chapters
        for chapter in content_list:
            for line in chapter.split('\n'):
                if line.startswith('# Chapter'):
                    story.append(Paragraph(line, self.styles['Heading1']))
                else:
                    story.append(Paragraph(line, self.styles['BodyText']))
            story.append(Spacer(1, 0.25 * inch))
            story.append(PageBreak())

        doc.build(story)
        print(f"PDF created: {filename}")
        return filename


    def save_book_as_epub(self, title, content_list, cover_image_path):
        filename = f"{title.replace(' ', '_')}.epub"
        book = epub.EpubBook()
        book.set_identifier('id123456')
        book.set_title(title)
        book.set_language('en')
        book.add_author('AI Author')

        if cover_image_path:
            # Add the cover image as the first item in the EPUB
            with open(cover_image_path, 'rb') as f:
                book.set_cover("cover.jpg", f.read())

            # Add the Title page to the EPUB (title above cover image)
            title_page = epub.EpubHtml(title="Cover", file_name="cover.xhtml", lang='en')
            title_page.content = f'''
            <html>
                <body>
                    <h1 style="text-align: center;">{title}</h1>
                    <img src="cover.jpg" alt="Cover Image" style="width:100%; height:auto;" />
                </body>
            </html>'''
            book.add_item(title_page)

        # Table of Contents and Chapters
        chapters = []
        for i, content in enumerate(content_list, 1):
            chapter = epub.EpubHtml(title=f"Chapter {i}", file_name=f"chap_{i}.xhtml", lang='en')
            content_lines = content.split('\n')[1:]  # Remove the Chapter title
            chapter_content = '<p>' + '</p><p>'.join(content_lines) + '</p>'
            chapter.content = f"<h1>Chapter {i}</h1>{chapter_content}"
            book.add_item(chapter)
            chapters.append(chapter)

        # Define Table of Contents
        book.toc = tuple(chapters)
        book.spine = ['nav'] + chapters
        book.add_item(epub.EpubNcx())
        book.add_item(epub.EpubNav())

        # Write the EPUB file
        epub.write_epub(filename, book)
        print(f"EPUB created: {filename}")
        return filename



# Widgets
language_dropdown = widgets.Dropdown(
    options=["English (en)", "Spanish (es)", "French (fr)", "German (de)", "Custom"],
    value="English (en)",
    description="Language:"
)

custom_language_input = widgets.Text(
    value="",
    placeholder="ISO 639-1 code (e.g., 'bn')",
    description="Custom:",
    disabled=True
)

title_input = widgets.Text(value="The Quantum Chronicles", description="Title:")
description_input = widgets.Textarea(
    value="A sci-fi adventure across dimensions.",
    description="Description:",
    layout={'width': '400px', 'height': '100px'}
)
pages_input = widgets.IntSlider(value=50, min=10, max=500, step=10, description="Pages:")
format_dropdown = widgets.Dropdown(options=["PDF", "EPUB", "Both"], value="PDF", description="Format:")
generate_button = widgets.Button(description="Generate Book", button_style="success")

def on_language_change(change):
    custom_language_input.disabled = change['new'] != "Custom"

language_dropdown.observe(on_language_change, names='value')

def generate_book_clicked(b):
    book_gen = BookGenerator(openai.api_key)
    title = title_input.value
    description = description_input.value
    target_pages = pages_input.value
    language = custom_language_input.value if language_dropdown.value == "Custom" else language_dropdown.value.split("(")[1][:-1]

    book_content, cover_image_path = book_gen.generate_book(title, description, target_pages, language)

    from google.colab import files
    if format_dropdown.value in ["PDF", "Both"]:
        pdf_file = book_gen.save_book_as_pdf(title, book_content, cover_image_path)
        files.download(pdf_file)
    if format_dropdown.value in ["EPUB", "Both"]:
        epub_file = book_gen.save_book_as_epub(title, book_content, cover_image_path)
        files.download(epub_file)

generate_button.on_click(generate_book_clicked)

display(language_dropdown, custom_language_input, title_input, description_input, pages_input, format_dropdown, generate_button)

Dropdown(description='Language:', options=('English (en)', 'Spanish (es)', 'French (fr)', 'German (de)', 'Cust…

Text(value='', description='Custom:', disabled=True, placeholder="ISO 639-1 code (e.g., 'bn')")

Text(value='The Quantum Chronicles', description='Title:')

Textarea(value='A sci-fi adventure across dimensions.', description='Description:', layout=Layout(height='100p…

IntSlider(value=50, description='Pages:', max=500, min=10, step=10)

Dropdown(description='Format:', options=('PDF', 'EPUB', 'Both'), value='PDF')

Button(button_style='success', description='Generate Book', style=ButtonStyle())

Generating 'The Quantum Chronicles' (en, ~50 pages, ~6 chapters)
Chapter 1...
Chapter 2...
Chapter 3...
Chapter 4...
Chapter 5...
Chapter 6...
PDF created: The_Quantum_Chronicles.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>